In [6]:
from bs4 import BeautifulSoup
import requests
import re 

In [ ]:
# This is the piece of code to scrape all of the urls - takes about 7mins without scraping

# full_list_url = []

# for i in range(1,333):
#     base_url = f"https://www.immoweb.be/en/search/house/for-sale?countries=BE&page={i}&orderBy=relevance"
#     r = requests.get(base_url)
#     soup = BeautifulSoup(r.content, "html.parser")
    
#     for elem in soup.find_all("a", attrs={"class":"card__title-link"}):
#          full_list_url.append(elem.get('href'))
#          with open("full_list.txt", "a") as f:
#             f.write(elem.get('href'))
#             f.write('\n')
            
# print("Written to text file Successfully!")
# print(len(full_list_url))

In [3]:
# TEST THREADING
import requests
from bs4 import BeautifulSoup
import threading
import time

# Function to scrape URLs
def scrape_urls(page_num):
    base_url = f"https://www.immoweb.be/en/search/house/for-sale?countries=BE&page={page_num}&orderBy=relevance"
    r = requests.get(base_url)
    soup = BeautifulSoup(r.content, "html.parser")
    
    urls = []
    for elem in soup.find_all("a", attrs={"class": "card__title-link"}):
        urls.append(elem.get('href'))
        
    # Save URLs to file - full_list.txt (local storage)
    with open("full_list.txt", "a") as f:
        for url in urls:
            f.write(url + '\n')
    return urls

def thread_scraping():
    full_list_url = []
    num_pages = 333

    # Create a list to store threads
    threads = []
    start_time = time.time()  # Start timer
    
    # Create and start threads
    for i in range(1, num_pages + 1):
        t = threading.Thread(target=lambda: full_list_url.extend(scrape_urls(i)))
        threads.append(t)
        t.start()

    # Wait for all threads to complete and then join
    for t in threads:
        t.join()

    end_time = time.time()  # Stop timer
    execution_time = end_time - start_time

    print("Scraping completed!")
    print("Total URLs scraped:", len(full_list_url))
    print("Total time:", execution_time, "seconds")


thread_scraping()

Scraping completed!
Total URLs scraped: 19980
Total time: 58.4198853969574 seconds


In [34]:
import requests
from bs4 import BeautifulSoup as soup
import re
import json

#https://www.immoweb.be/en/classified/house/for-sale/libin/6890/10657263        

url = "https://www.immoweb.be/en/classified/house/for-sale/libin/6890/10657263"
house_page = requests.get(url)
house_page = soup(house_page.text, 'html.parser')

regex = r"window.classified = (\{.*\})"

script = house_page.find('div',attrs={"id":"main-container"}).script.text
script = re.findall(regex, script)
 
script = json.loads(script[0])
#print(script['property']['location']['locality'])
script

{'id': 10657263,
 'cluster': None,
 'customers': [{'id': 3151988,
   'type': 'AGENCY',
   'email': 'honesty_bastogne_578@importfrommedia.be',
   'logoUrl': 'https://static.immoweb.be/logos/3151988.gif?cache=2022530706833',
   'phoneNumber': '+32 61 29 24 14',
   'mobileNumber': None,
   'name': 'Honesty Bertrix - 7 bureaux proches de chez vous',
   'website': 'http://honestybastogne.be/',
   'location': {'country': 'Belgium',
    'region': None,
    'province': 'Luxembourg',
    'district': 'Neufchâteau',
    'locality': 'Bertrix',
    'postalCode': '6880',
    'street': 'rue des Corettes',
    'number': None,
    'box': None,
    'propertyName': None,
    'floor': None,
    'latitude': 50,
    'longitude': 5.71,
    'distance': None,
    'approximated': None,
    'regionCode': None,
    'type': None,
    'hasSeaView': None,
    'pointsOfInterest': None,
    'placeName': None},
   'ipiNo': '504.686',
   'isOwner': True,
   'contactHoursMobile': 'anytime',
   'contactHoursLandline': 'an

In [31]:
final_dictionary = {}

#- Number of rooms
final_dictionary['number_rooms'] = script['property']['bedroomCount']
# living area
final_dictionary['living_area'] = script['property']['netHabitableSurface']
# Fully equipped kitchen (Yes/No)
final_dictionary['kitchen'] = script['property']['kitchen']['type']
# NOT INSTALLED / INSTALLED 
# Furnished (Yes/No)
final_dictionary['furnished'] = script['transaction']['sale']['isFurnished']
# Open fire (Yes/No)
final_dictionary['fireplace'] = script['property']['fireplaceCount']
# Terrace (Yes/No)
final_dictionary['terrace'] = script['property']['hasTerrace']
# If yes: Area
final_dictionary['terrace_area'] = script['property']['terraceSurface']

In [32]:
final_dictionary

{'number_rooms': 3,
 'living_area': 124,
 'kitchen': 'NOT_INSTALLED',
 'furnished': False,
 'fireplace': None,
 'terrace': True,
 'terrace_area': 8}